<a href="https://colab.research.google.com/github/MasterBeard/EigenCluster/blob/main/EigenCluster_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import yfinance as yf

# Define index tickers
index_tickers = {
    'SPX': '^GSPC',     # S&P 500
    'IXIC': '^IXIC',    # NASDAQ Composite
    'HSI': '^HSI',      # Hang Seng Index
    'DJI': '^DJI',      # Dow Jones Industrial Average
    'FCHI': '^FCHI',    # CAC 40
    'DAXI': '^GDAXI',   # DAX
    'N225': '^N225',    # Nikkei 225
    'KS11': '^KS11',    # KOSPI
    'SENSEX': '^BSESN', # BSE Sensex
    'STOXX50': '^STOXX50E',  # EURO STOXX 50
}

date_ranges = {
    'train': ("2000-01-01", "2009-12-31"),
    'val': ("2010-01-02", "2010-12-31")
}

# Store feature matrices and labels for each split (including norm_features)
data_splits = {split: {'features': [], 'norm_features': [], 'labels': []} for split in date_ranges}

# Window size
window_size = 11

# Fetch and process data for each time period
for split, (start_date, end_date) in date_ranges.items():
    index_data = {}
    for name, ticker in index_tickers.items():
        if ticker == '000001.SS':
            index_data[name] = yf.Ticker(ticker).history(start=start_date, end=end_date, auto_adjust=True)
        else:
            index_data[name] = yf.download(ticker, start=start_date, end=end_date, auto_adjust=True)

    # Create feature vectors and labels for each index
    for index_name, data in index_data.items():
        if data.empty:
            continue

        # Fix multi-level column names (only happens with Ticker().history())
        if isinstance(data.columns, pd.MultiIndex):
            data.columns = data.columns.get_level_values(0)

        open_values = data['Open'].dropna().values
        close_values = data['Close'].dropna().values
        low_values = data['Low'].dropna().values
        high_values = data['High'].dropna().values

        for start in range(len(data) - window_size + 1):
            open_row = open_values[start:start + window_size]
            low_row = low_values[start:start + window_size]
            high_row = high_values[start:start + window_size]
            close_row = close_values[start:start + window_size]

            # Build open-low-high-close-volume feature vector (not normalized)
            combined = np.array([
                val for i in range(window_size)
                for val in (open_row[i], low_row[i], high_row[i], close_row[i])
            ])

            # Normalized feature vector
            norm_combined = np.array([
                (open_row[i] / close_row[-2],
                low_row[i] / close_row[-2],
                high_row[i] / close_row[-2],
                close_row[i] / close_row[-2])
                for i in range(window_size)
            ]).flatten()

            label = 1 if close_row[-1] > close_row[-2] else 0

            # Store results
            data_splits[split]['features'].append(combined)
            data_splits[split]['norm_features'].append(norm_combined)
            data_splits[split]['labels'].append(label)

# Convert to NumPy arrays
train_features = np.array(data_splits['train']['features'])
train_norm_features = np.array(data_splits['train']['norm_features'])
train_labels = np.array(data_splits['train']['labels'])

val_features = np.array(data_splits['val']['features'])
val_norm_features = np.array(data_splits['val']['norm_features'])
val_labels = np.array(data_splits['val']['labels'])

# Output shapes of each set
print(f"Train features shape: {train_features.shape}")
print(f"Train norm_features shape: {train_norm_features.shape}")
print(f"Train labels shape: {train_labels.shape}")

print(f"Validation features shape: {val_features.shape}")
print(f"Validation norm_features shape: {val_norm_features.shape}")
print(f"Validation labels shape: {val_labels.shape}")

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

Train features shape: (23092, 44)
Train norm_features shape: (23092, 44)
Train labels shape: (23092,)
Validation features shape: (2410, 44)
Validation norm_features shape: (2410, 44)
Validation labels shape: (2410,)


In [2]:
import numpy as np
from sklearn.cluster import KMeans

# ============================================
# Utility functions
# ============================================
def extract_normalized_daily_ohlc(matrices, epsilon=1e-6):
    """
    Each window is normalized by its 5th-to-last value

    Args:
        matrices: Input feature matrices
        epsilon: Small value to avoid division by zero

    Returns:
        Normalized window data as numpy array
    """
    all_windows = []

    for vec in matrices:
        reshaped = vec.reshape(-1, 4)  # shape: (11, 4)
        for i in range(1, reshaped.shape[0] - 10 + 1):  # Total 7 windows
            window = reshaped[i:i + 10]  # shape: (5, 4)
            flat = window.flatten()      # shape: (20,)

            denominator = flat[-5]  # 5th-to-last value (flat[15], day 4's close)
            if abs(denominator) < epsilon:
                denominator = 1.0  # Avoid division by zero

            normalized = flat / denominator
            all_windows.append(normalized)

    return np.array(all_windows)  # shape: [N_samples × 7, 20]

# Process training and validation data
train_days = extract_normalized_daily_ohlc(train_features)
val_days = extract_normalized_daily_ohlc(val_features)

In [3]:
from sklearn.decomposition import PCA, NMF, TruncatedSVD

# 1. Principal Component Analysis (PCA) - variance maximization
# Initialize PCA with all components (same dimension as input)
pca = PCA(n_components=train_days.shape[1])

# Transform training and validation data using PCA
train_1d = pca.fit_transform(train_days)  # Fit and transform training data
val_1d = pca.transform(val_days)          # Transform validation data (using same PCA model)

# Create features for clustering:
# Combine mean of sine values with vector norm as a new feature
train_theta = (np.mean(np.sin(train_1d), axis=1) * np.linalg.norm(train_1d, axis=1)).reshape(-1, 1)
val_theta = (np.mean(np.sin(val_1d), axis=1) * np.linalg.norm(val_1d, axis=1)).reshape(-1, 1)

# Perform K-means clustering with 15 clusters
kmeans = KMeans(n_clusters=15, random_state=0)  # Using fixed random state for reproducibility
train_clusters = kmeans.fit_predict(train_theta)  # Fit on training data
val_clusters = kmeans.predict(val_theta)         # Predict on validation data

# Get cluster centers (shape: (n_clusters, 1) since we're using 1D features)
centers = kmeans.cluster_centers_

In [4]:
import numpy as np

# Get all unique cluster centers (sorted)
unique_clusters = np.unique(train_clusters)  # All cluster indices

# Store class 1 ratio and sample count for each cluster
cluster_info = []

# Calculate statistics for each cluster
for cluster in unique_clusters:
    # Create mask for current cluster
    cluster_mask = (train_clusters == cluster)
    cluster_labels = train_labels[cluster_mask]

    # Calculate class statistics
    n_total = len(cluster_labels)
    n_class1 = np.sum(cluster_labels == 1)
    class1_ratio = n_class1 / n_total

    # Store cluster information
    cluster_info.append({
        "cluster": cluster,
        "class1_ratio": class1_ratio,
        "n_class1": n_class1,
        "n_class0": n_total - n_class1,
        "n_total": n_total
    })

# Custom sorting key function
def sort_key(info):
    """
    Sorting key function that prioritizes:
    1. Clusters with >50% class 1 (sorted by n_class1 ascending)
    2. Other clusters (sorted by n_class0 descending)
    """
    ratio = info["class1_ratio"]
    if ratio > 0.5:
        # For >50% class 1: sort by n_class1 ascending (smaller samples rank higher)
        return (0, info["n_class1"])  # 0 indicates high priority group
    else:
        # For ≤50% class 1: sort by n_class0 descending (smaller samples rank lower)
        return (1, -info["n_class0"])  # 1 indicates low priority group

# Sort clusters using custom rules
sorted_clusters = sorted(cluster_info, key=sort_key)

# Print results
print("Clusters sorted by custom rules:")
for info in sorted_clusters:
    cluster = info["cluster"]
    ratio = info["class1_ratio"]
    n_class0 = info["n_class0"]
    n_class1 = info["n_class1"]
    n_total = info["n_total"]

    print(f"Cluster {cluster}:")
    print(f"  Class 0 count: {n_class0}")
    print(f"  Class 1 count: {n_class1}")
    print(f"  Class 1 ratio: {ratio:.2%}")
    print("---")

Clusters sorted by custom rules:
Cluster 1:
  Class 0 count: 5
  Class 1 count: 17
  Class 1 ratio: 77.27%
---
Cluster 12:
  Class 0 count: 18
  Class 1 count: 39
  Class 1 ratio: 68.42%
---
Cluster 2:
  Class 0 count: 54
  Class 1 count: 81
  Class 1 ratio: 60.00%
---
Cluster 6:
  Class 0 count: 93
  Class 1 count: 151
  Class 1 ratio: 61.89%
---
Cluster 5:
  Class 0 count: 208
  Class 1 count: 342
  Class 1 ratio: 62.18%
---
Cluster 8:
  Class 0 count: 539
  Class 1 count: 661
  Class 1 ratio: 55.08%
---
Cluster 0:
  Class 0 count: 1493
  Class 1 count: 1693
  Class 1 ratio: 53.14%
---
Cluster 14:
  Class 0 count: 5405
  Class 1 count: 5865
  Class 1 ratio: 52.04%
---
Cluster 9:
  Class 0 count: 1947
  Class 1 count: 1919
  Class 1 ratio: 49.64%
---
Cluster 4:
  Class 0 count: 774
  Class 1 count: 752
  Class 1 ratio: 49.28%
---
Cluster 11:
  Class 0 count: 328
  Class 1 count: 311
  Class 1 ratio: 48.67%
---
Cluster 7:
  Class 0 count: 134
  Class 1 count: 109
  Class 1 ratio: 44.86

In [5]:
# Keep middle 36 features (remove first 4 and last 4 columns)
train_x = train_norm_features[:, 4:40]  # Shape: [n_samples, 36]
val_x = val_norm_features[:, 4:40]     # Shape: [n_samples, 36]

# Use cluster assignments as labels
y_train = train_clusters
y_val = val_clusters

# Verify new shapes
print("Train features new shape:", train_x.shape)
print("Validation features new shape:", val_x.shape)

import numpy as np
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

# 1. Define models with consistent configurations
models = {
    "RandomForest": RandomForestClassifier(
        n_estimators=150,
        max_depth=None,              # No maximum depth
        min_samples_split=2,         # Minimum samples to split node
        min_samples_leaf=2,          # Minimum samples at leaf node
        max_features='sqrt',         # Features to consider at each split
        random_state=42              # For reproducibility
    ),
    "XGBoost": XGBClassifier(
        n_estimators=100,
        max_depth=5,                 # Maximum tree depth
        learning_rate=0.1,           # Boosting learning rate
        subsample=1.0,              # Subsample ratio of training instances
        colsample_bytree=0.8,        # Subsample ratio of features
        random_state=42,
        # 🔴 Key parameters for multi-class classification
        objective='multi:softmax',   # Multiclass objective function
        num_class=len(np.unique(y_train)),  # Number of classes
        eval_metric='mlogloss',      # Multiclass evaluation metric
        early_stopping_rounds=10     # Early stopping rounds
    )
}

# 2. Unified training process (with explicit validation set)
for model_name, model in models.items():
    print(f"\n=== Training {model_name} ===")

    if model_name == "XGBoost":
        # XGBoost with early stopping on validation set
        model.fit(
            train_x, y_train,
            eval_set=[(val_x, y_val)],
            verbose=False
        )
    else:
        # Standard fit for other models
        model.fit(train_x, y_train)

# 3. Evaluation function for cluster analysis
def evaluate_cluster_range(cluster_indices, true_labels, pred_labels):
    """
    Evaluate performance on specific clusters

    Args:
        cluster_indices: List of cluster indices to evaluate
        true_labels: Ground truth labels
        pred_labels: Predicted cluster assignments

    Returns:
        Tuple of (mean accuracy, sample count) or (None, 0) if no samples
    """
    mask = np.isin(pred_labels, cluster_indices)
    y_subset = true_labels[mask]
    return (np.mean(y_subset), len(y_subset)) if len(y_subset) > 0 else (None, 0)

Train features new shape: (23092, 36)
Validation features new shape: (2410, 36)

=== Training RandomForest ===

=== Training XGBoost ===


In [6]:
import numpy as np
import pandas as pd
import zipfile
import os
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')
successful_stocks = 0  # Counter for successfully processed stocks

# File path configuration
zip_path = '/content/drive/My Drive/SP500_data2010-2020.zip'
extract_dir = '/content/SP500_data2010-2020'

# Extract zip file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

def read_sp500_csv(filepath):
    """
    Read S&P 500 CSV file and filter data for 2011-2020 period

    Args:
        filepath: Path to the CSV file

    Returns:
        DataFrame containing OHLC data for the specified period
    """
    # Read CSV with multi-level headers and drop the second level
    df = pd.read_csv(filepath, header=[0,1], index_col=0, parse_dates=True)
    df.columns = df.columns.droplevel(1)

    # Keep only required columns
    required_cols = ['Open', 'High', 'Low', 'Close']
    missing_cols = [col for col in required_cols if col not in df.columns]
    if missing_cols:
        raise ValueError(f"Missing required columns: {missing_cols}")

    # Set date range filter (2011-2020)
    start_date = pd.Timestamp("2011-01-01", tz="UTC") if df.index.tz else pd.Timestamp("2011-01-01")
    end_date = pd.Timestamp("2020-12-31", tz="UTC") if df.index.tz else pd.Timestamp("2020-12-31")

    # Filter data by date range
    mask = (df.index >= start_date) & (df.index <= end_date)
    df = df.loc[mask]

    return df[required_cols]

# Main processing pipeline
window_size = 11  # Size of the rolling window
test_features1 = []  # Raw features storage
test_norm_features1 = []  # Normalized features storage
test_labels1 = []  # Labels storage (1=up, 0=down)

# Get all CSV files in directory
csv_files = sorted([f for f in os.listdir(extract_dir) if f.endswith('.csv')])
print(f"Found {len(csv_files)} S&P 500 constituent stock files")

for csv_file in csv_files[:]:  # Process all files
    try:
        filepath = os.path.join(extract_dir, csv_file)

        # Read and process CSV file
        ohlc = read_sp500_csv(filepath)
        if len(ohlc['Open'].values) != 0:
            successful_stocks += 1

        # Skip if no data in date range
        if len(ohlc) == 0:
            print(f"{csv_file} has no data in 2011-2020 period")
            continue

        # Extract OHLC values
        open_vals = ohlc['Open'].values
        high_vals = ohlc['High'].values
        low_vals = ohlc['Low'].values
        close_vals = ohlc['Close'].values

        # Create rolling window features
        for start in range(len(ohlc) - window_size):
            window_open = open_vals[start:start+window_size]
            window_high = high_vals[start:start+window_size]
            window_low = low_vals[start:start+window_size]
            window_close = close_vals[start:start+window_size]

            # Create feature vector (Open, Low, High, Close order)
            feature_vec = np.array([
                val for i in range(window_size)
                for val in (window_open[i], window_low[i], window_high[i], window_close[i])
            ])

            # Create label (1 if price increased, 0 otherwise)
            label = 1 if window_close[-1] > window_close[-2] else 0

            test_features1.append(feature_vec)
            test_norm_features1.append(feature_vec/window_close[-2])  # Normalized by previous close
            test_labels1.append(label)

    except Exception as e:
        print(f"Error processing {csv_file}: {str(e)}")
        continue

# Convert to numpy arrays
if test_features1:  # Check if any data was processed
    test_features1 = np.array(test_features1)
    test_norm_features1 = np.array(test_norm_features1)
    test_labels1 = np.array(test_labels1)
else:
    test_features1 = np.array([])
    test_labels1 = np.array([])
    test_norm_features1 = np.array([])

# Output results
print("\nTest set generation results:")
if len(test_features1) > 0:
    print(f"Feature matrix shape: {test_features1.shape}")
    print(f"Normalized feature matrix shape: {test_norm_features1.shape}")
    print(f"Successfully processed stocks: {successful_stocks}")
    print(f"Label distribution - Up: {np.mean(test_labels1):.1%}, Down: {1-np.mean(test_labels1):.1%}")
else:
    print("No features generated - please check input files or date range")

Mounted at /content/drive
Found 501 S&P 500 constituent stock files
AMTM.csv has no data in 2011-2020 period
CEG.csv has no data in 2011-2020 period
GEHC.csv has no data in 2011-2020 period
GEV.csv has no data in 2011-2020 period
KVUE.csv has no data in 2011-2020 period
SOLV.csv has no data in 2011-2020 period
SW.csv has no data in 2011-2020 period
VLTO.csv has no data in 2011-2020 period

Test set generation results:
Feature matrix shape: (1176149, 44)
Normalized feature matrix shape: (1176149, 44)
Successfully processed stocks: 493
Label distribution - Up: 52.2%, Down: 47.8%


In [7]:
# Get test data for the current index
test_x = test_norm_features1[:, 4:40]  # Select middle 36 features (remove first 4 and last 4)
test_labels = test_labels1  # Ground truth labels

# Evaluate each model
for model_name, model in models.items():
    print(f"\nModel: {model_name}")

    # Make predictions on test set
    y_pred = model.predict(test_x)

    # Store predictions for evaluation
    test_pred1 = y_pred

    # Evaluate top performing clusters
    print("Top Cluster Evaluation:")
    for n in range(1, 8):  # Evaluate top 1-7 clusters
        # Get cluster IDs of top n clusters
        clusters = sorted_clusters[:n]
        clusters = [item['cluster'] for item in clusters]

        # Evaluate performance on these clusters
        acc, count = evaluate_cluster_range(clusters, test_labels, test_pred1)

        if acc is not None:
            # Print accuracy and sample count for these clusters
            print(f"Top {n} clusters: Acc={acc:.4f}, Samples={count}")
        else:
            print(f"Top {n} clusters: No samples")

    # Evaluate bottom performing clusters
    print("\nBottom Cluster Evaluation:")
    for n in range(1, 8):  # Evaluate bottom 1-7 clusters
        # Get cluster IDs of bottom n clusters
        clusters = sorted_clusters[-n:]
        clusters = [item['cluster'] for item in clusters]

        # Evaluate performance on these clusters
        acc, count = evaluate_cluster_range(clusters, test_labels, test_pred1)

        if acc is not None:
            # Print inverse accuracy (1-acc) since these are worst performers
            print(f"Bottom {n} clusters: Acc={(1-acc):.4f}, Samples={count}")
        else:
            print(f"Bottom {n} clusters: No samples")


Model: RandomForest
Top Cluster Evaluation:
Top 1 clusters: Acc=0.5922, Samples=3136
Top 2 clusters: Acc=0.5746, Samples=7007
Top 3 clusters: Acc=0.5619, Samples=15735
Top 4 clusters: Acc=0.5511, Samples=29829
Top 5 clusters: Acc=0.5461, Samples=56120
Top 6 clusters: Acc=0.5401, Samples=109646
Top 7 clusters: Acc=0.5369, Samples=248244

Bottom Cluster Evaluation:
Bottom 1 clusters: Acc=0.6041, Samples=2900
Bottom 2 clusters: Acc=0.5626, Samples=7385
Bottom 3 clusters: Acc=0.5354, Samples=16751
Bottom 4 clusters: Acc=0.5158, Samples=35170
Bottom 5 clusters: Acc=0.5036, Samples=75567
Bottom 6 clusters: Acc=0.4986, Samples=166374
Bottom 7 clusters: Acc=0.4907, Samples=370209

Model: XGBoost
Top Cluster Evaluation:
Top 1 clusters: Acc=0.6249, Samples=2141
Top 2 clusters: Acc=0.5714, Samples=6683
Top 3 clusters: Acc=0.5603, Samples=15192
Top 4 clusters: Acc=0.5521, Samples=29607
Top 5 clusters: Acc=0.5464, Samples=54948
Top 6 clusters: Acc=0.5405, Samples=107763
Top 7 clusters: Acc=0.5370,